In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json


In [2]:
import io
# import pandas as pd
# from google.colab import files
from collections import OrderedDict
import inspect as inspector
import sys
import argparse
# import json
import time
import random
import hashlib

In [3]:
!pip install -q "tensorflow-text"

import requests
import tensorflow as tf
import tensorflow_text as tf_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 15.8 MB/s eta 0:00:00


In [4]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 112.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.8 MB/s eta 0:00:00


In [5]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

!pip install transformers #==2.9.0
!pip install torch  #==2.0.0

# !pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.1.0-py3-none-any.whl

!pip install accelerate
# !pip install transformers==2.9.0 
!pip install pytorch_lightning  #==0.7.5
# !pip install datsets transformers[sentencepiece]
!pip install sentencepiece

!pip install Sentencepiece
# !pip install transformers

!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 71.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.5 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=436140a8d96af085e082b531e602a6df9247e0da17d38da3057dc48bfe2a20eb
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-pytho

In [6]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)
df_validation = pd.read_json('./validation.jsonl', lines=True)

In [7]:
def concatenate_strings(lst):
    separator = ' '
    return separator.join(lst)

In [8]:
df['targetParagraphs'] = df['targetParagraphs'].apply(concatenate_strings)
df['postText'] = df['postText'].apply(concatenate_strings)
df['spoiler'] = df['spoiler'].apply(concatenate_strings)
df['tags'] = df['tags'].apply(concatenate_strings)

In [9]:
df_validation['targetParagraphs'] = df_validation['targetParagraphs'].apply(concatenate_strings)
df_validation['postText'] = df_validation['postText'].apply(concatenate_strings)
df_validation['spoiler'] = df_validation['spoiler'].apply(concatenate_strings)
df_validation['tags'] = df_validation['tags'].apply(concatenate_strings)

In [10]:
filter_phrase = df['tags'] == "passage"
df_filtered = df[filter_phrase]
# print (df_filtered)

In [11]:
filter_phrase_validation = df_validation['tags'] == "passage"
df_filtered_validation = df_validation[filter_phrase_validation]
# print (df_filtered_validation)

In [12]:
def generate_new_df(df):

    new_df = pd.DataFrame({'tags': df['tags'],'postText':df['postText'],'paragraph' : df['targetParagraphs'] ,'spoiler': df['spoiler']})
    return new_df
    

In [13]:

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



# sklearn

from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



# Utility

import re
import string
import pickle

In [14]:
wordLemm = WordNetLemmatizer()
def processed_postText(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    
    return ' '.join(tokens)

In [15]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [16]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
new_df = generate_new_df(df_filtered)

In [18]:
new_df_validation = generate_new_df(df_filtered_validation)

In [19]:
 new_df['paragraph'] = new_df['paragraph'].apply(lambda x: processed_postText(x))
 new_df['postText'] = new_df['postText'].apply(lambda x: processed_postText(x))
# print('Text Preprocessing complete.')

In [20]:
 new_df_validation['paragraph'] = new_df_validation['paragraph'].apply(lambda x: processed_postText(x))
 new_df_validation['postText'] = new_df_validation['postText'].apply(lambda x: processed_postText(x))

In [22]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 28.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=3c052945b527aff42fffcff82074a748de0b47a33afe6049e44007ae7be53646
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [23]:
import torch

In [24]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [25]:
from datetime import datetime

In [26]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_wiki_lingua",
        config={
            "per_device_train_batch_size": 4
            # "learning_rate": training_args.learning_rate
            # "dataset": "wiki_lingua " + "english",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + "english" + "_" + current_time

wandb: Currently logged in as: pradhaneva94 (evapradhan). Use `wandb login --relogin` to force relogin


In [ ]:
#FINE TUNING START

In [27]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00


In [28]:
from datasets import Dataset
from datasets import dataset_dict

In [29]:
import transformers
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, load_dataset, load_from_disk, load_metric
import numpy as np
import nltk
nltk.download('punkt')
import torch
import pandas as pd

from transformers import (TrainingArguments, Trainer, EarlyStoppingCallback)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
# max_input = 512
# max_target = 128
# batch_size = 3
model_checkpoints = "t5-small"

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

In [32]:
# input_text = "Summarize:" + "condition: " + new_df['postText'] + " paragraph: " + new_df["paragraph"]
input_text = "Summarize:" + new_df["paragraph"]
target_text = new_df["spoiler"]


data_to_process_train = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [33]:
data_to_process_train

,input_ids,labels
0,Summarize:it ’ ll be just like old times this ...,how about that morning we go throw?
5,Summarize:one of the biggest surprise announce...,"Apple says that if AirPods are lost or stolen,..."
6,Summarize:june 14th 2016 33k shares they may l...,"""The more good games I had in them, the more I..."
7,Summarize:it was cold and very foggy the tempe...,rainbow colours in the sky and a halo spanning...
8,Summarize:55 say yes cheers to your health all...,Red wine is clearly the drink of choice if you...
...,...,...
3189,Summarize:what women want in a man has been re...,almost every other part of a man is considered...
3193,Summarize:if youre a vegetarian we have some g...,may cut your risk for colorectal cancer -- the...
3194,Summarize:if you see a purple butterfly sticke...,one of her unborn twins had a condition called...
3195,Summarize:a long time ago in a galaxy far far ...,it hasn’t necessarily taken the wind out of Yo...


In [34]:
df1_dict = data_to_process_train.to_dict(orient = 'list')

In [35]:
dict_train_ = Dataset.from_dict(df1_dict)

In [36]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1274
})

In [37]:
# input_text = "Summarize:" + "condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
input_text = "Summarize:" + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_validation = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [38]:
data_to_process_train

,input_ids,labels
0,Summarize:it ’ ll be just like old times this ...,how about that morning we go throw?
5,Summarize:one of the biggest surprise announce...,"Apple says that if AirPods are lost or stolen,..."
6,Summarize:june 14th 2016 33k shares they may l...,"""The more good games I had in them, the more I..."
7,Summarize:it was cold and very foggy the tempe...,rainbow colours in the sky and a halo spanning...
8,Summarize:55 say yes cheers to your health all...,Red wine is clearly the drink of choice if you...
...,...,...
3189,Summarize:what women want in a man has been re...,almost every other part of a man is considered...
3193,Summarize:if youre a vegetarian we have some g...,may cut your risk for colorectal cancer -- the...
3194,Summarize:if you see a purple butterfly sticke...,one of her unborn twins had a condition called...
3195,Summarize:a long time ago in a galaxy far far ...,it hasn’t necessarily taken the wind out of Yo...


In [39]:
df2_dict = data_to_process_validation.to_dict(orient = 'list')

In [40]:
dict_validation_ = Dataset.from_dict(df2_dict)

In [41]:
dict_validation_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 322
})

In [ ]:
dict_validation_["input_ids"]

In [44]:
# input_text = "Summarize:" + "condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
input_text = "Summarize:" + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_test = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [45]:
data_to_process_test

,input_ids,labels
0,Summarize:five nights at freddy ’ s creator sc...,some of the plot elements are so disturbing th...
4,Summarize:petapixel is one of my favorite blog...,a man who swallowed a 64GB microSD card and th...
7,Summarize:from reading jk rowling ‘ s harry po...,McGonagall was appointed as Dumbledore’s assis...
8,Summarize:in the modern blockbuster world post...,All the scenes are actually in the movie
9,Summarize:huge fan of all that drama on the hi...,"""I had fake relationships, fake fights. I don'..."
...,...,...
792,Summarize:have you ever come across an abandon...,"This was clearly a crack house, and he was sta..."
793,Summarize:reuters for years economists have be...,China is too quick to rebalance its services s...
794,Summarize:it was a routine walk for one colora...,the plant the dog was chewing on was deadly wa...
795,Summarize:instead of encountering a mound of d...,The video below shows the stunned cleaner init...


In [46]:
df3_dict = data_to_process_test.to_dict(orient = 'list')

In [47]:
dict_test_ = Dataset.from_dict(df3_dict)

In [48]:
dict_test_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 322
})

In [ ]:
# df1_dict = data_to_process_train.to_dict(orient = 'list')
# df2_dict = data_to_process_validation.to_dict(orient = 'list')
# df3_dict = data_to_process_test.to_dict(orient = 'list')


# dict_train_ = Dataset.from_dict(df1_dict)
# dict_validation_ = Dataset.from_dict(df2_dict)
# dict_test_ = Dataset.from_dict(df3_dict)

In [51]:
dict_all_2 = dataset_dict.DatasetDict({"train":dict_train_, "validation":dict_validation_, "test": dict_test_})

In [52]:
dict_all_2

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1274
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 322
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 322
    })
})

In [53]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1274
})

In [43]:
def preprocess_data(data_to_process):
    data_to_process_input = data_to_process['input_ids']
    data_to_process_target = data_to_process['labels']

    inp = tokenizer(data_to_process_input, padding="max_length", truncation=True, max_length= 1000).input_ids

    with tokenizer.as_target_tokenizer():
      targ = tokenizer(data_to_process_target, padding='max_length', truncation=True, max_length= 500).input_ids
    
    return {"input_ids": inp, "labels": targ}

In [44]:
train_dataset = dict_train_
test_dataset = dict_validation_

train_dataset_processed = train_dataset.map(preprocess_data, batched=True)
test_dataset_processed = test_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

In [ ]:
test_dataset_processed["input_ids"]

In [56]:
train_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1274
})

In [57]:
test_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 322
})

In [53]:
from transformers import pipeline
import requests
import pprint
import time
pp = pprint.PrettyPrinter(indent=14)

from transformers import T5Tokenizer, T5ForConditionalGeneration
# import wandb

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
#testing

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [48]:
#testing
# /content/drive/MyDrive/NLP_Projects/T5_finetuning__passage_results_conditional_all_sizelimited
model_path_saved = '/content/drive/MyDrive/NLP_Projects/T5_finetuning__passage_results_conditional_all_sizelimited/checkpoint-7000'
model_saved = T5ForConditionalGeneration.from_pretrained(model_path_saved)

# tokenizer = AutoTokenizer.from_pretrained(model_saved)

In [51]:
#training

model_name = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, num_labels = 4)

In [60]:
bleu_metric = load_metric("bleu")
meteor_metric = load_metric("meteor")
bertscore_metric = load_metric("bertscore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [61]:
# #out_1 = []
# #out_2 = []

# def compute_metrics(eval_pred):
#     logits_full, labels = eval_pred

#     #out_1.append(logits_full[0])
#     #out_2.append(logits_full[1])

#     predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
#     decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     decoded_preds_bleu = [pred.split() for pred in decoded_preds]
#     decoded_labels_bleu = [[pred.split()] for pred in decoded_labels]
    
#     bleu_score = bleu_metric.compute(predictions=decoded_preds_bleu, references= decoded_labels_bleu)
#     meteor_score = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
#     bertscore_score = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang = 'en')
    
#     return {"bleu": bleu_score["bleu"], "meteor": meteor_score["meteor"], "bertscore": np.mean(bertscore_score["f1"])}

In [74]:
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/NLP_Projects/T5_finetuning__passage_results_conditional_all_sizelimited",
    num_train_epochs=25,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    logging_steps=100,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    # predict_with_generate=True,
    fp16=True,
    # save_strategy="epoch"

)




In [75]:
#training

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=test_dataset_processed,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    # compute_metrics=compute_metrics
)

In [76]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.165500,0.137738
2,0.127900,0.117808
3,0.099700,0.102935
4,0.093000,0.094790
5,0.089200,0.087299
6,0.074900,0.080676
7,0.090800,0.075884
8,0.077700,0.072017
9,0.077700,0.067992
10,0.067800,0.066486


TrainOutput(global_step=7975, training_loss=0.09045889092089614, metrics={'train_runtime': 2928.9536, 'train_samples_per_second': 10.874, 'train_steps_per_second': 2.723, 'total_flos': 8419211673600000.0, 'train_loss': 0.09045889092089614, 'epoch': 25.0})

In [ ]:
metrics = trainer.evaluate()


In [ ]:
metrics

{'eval_loss': 0.05783680081367493,
 'eval_runtime': 9.6498,
 'eval_samples_per_second': 33.369,
 'eval_steps_per_second': 8.394,
 'epoch': 25.0}

In [49]:
# model_path_saved = '/content/drive/MyDrive/T5_finetuning__passage_results_conditional_all_sizelimited/checkpoint-7500'
# model_saved = T5ForConditionalGeneration.from_pretrained(model_path_saved)

In [51]:
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/T5_finetuning__passage_results_conditional_all_sizelimited_testing",
    num_train_epochs=25,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    logging_steps=100,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    # predict_with_generate=True,
    fp16=True,
    # save_strategy="epoch"

)

In [52]:
trainer_saved = Trainer(
    model=model_saved,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=test_dataset_processed,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

# predictions_output = trainer_saved.predict(test_dataset_processed.select(range(len(test_dataset_processed))))
# logits_full = predictions_output.predictions
# predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
# decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

# result_frame = pd.DataFrame()
# result_frame['actual_spoiler'] = test_dataset_processed["labels"]
# result_frame['gen_spoiler'] = decoded_preds

In [57]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples,
        padding="max_length",
        truncation=True,
        max_length=1000,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


    # predictions_output = trainer_saved.predict(test_samples)
    # logits_full = predictions_output.predictions
    # predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
    # decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
    # return decoded_preds
    # result_frame = pd.DataFrame()
    # result_frame['actual_spoiler'] = test_dataset_processed["labels"]
    # result_frame['gen_spoiler'] = decoded_preds

In [ ]:
train_dataset_processed["labels"]

In [63]:
# model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summaries_after_tuning_all = []

def generate_batch(lst, batch_size):
    """  Yields batch of specified size """
    for i in range(0, len(lst), 
                   batch_size):
        yield lst[i : i + batch_size]
        # summaries_after_tuning = generate_summary(test_samples, model)[1]
        # summaries_after_tuning_all.append(summaries_after_tuning)


# test_samples = dict_validation_.select(range())
# test_samples = generate_batch(dict_validation_["input_ids"], 16)
test_samples = generate_batch(dict_validation_["input_ids"], 16)

# # summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]

for batch in test_samples:
  # print(batch)
  for sample in batch:
    # print(sample)
    # print("\n")
    summaries_after_tuning = generate_summary(sample, model_saved)[1]
    summaries_after_tuning_all.append(summaries_after_tuning)
    # print(summaries_after_tuning)
    # print("\n")

In [ ]:
# dataframe = result_frame

In [ ]:
summaries_after_tuning_all

[['Einige der plot elements are so disturbing that they make him feel sick',
  'Ein Mann, der swallowed a 64GB microSD card and then pooped',
  'McGonagall was appointed as Dumbledore’s assistant in 1956, not',
  'Alle Szenen sind eigentlich in dem Film.',
  '"I had fake relationships, fake fights. I don\'t care anymore, I can',
  "Er'd eaten a peanut butter sandwich and wasn't aware of her peanut allergy.",
  "'She kicked her and got into a fight with her current boyfriend.",
  'Dunham picked the boy up and took him to a Subway to get something to ',
  'Aubrey has cerebral palsy, but she was neglected and abused by her biological',
  "Leider, there's not enough solid research out there on whether melatonin",
  "Kevin Williamson said he didn't want to write it.",
  'Sophie and Riley were considered "micro-preemies" and suffered a s',
  'Buss and Jackson announced that they were mutually ending their four-year engagement.',
  'In the video, an abstract, wooden sculpture clock appeared n

'Einige der plot elements are so disturbing that they make him feel sick.'

In [64]:
stories_list =[]
for i in range(len(summaries_after_tuning_all)):
  for j in range(len(summaries_after_tuning_all[i])):
    summary=summaries_after_tuning_all[i][j]
    stories_list.append(summary)


print(stories_list)

['Five nights at freddy’s sister location is scheduled to launch this coming Friday, October', 'The story is written in the first person by John Doe, who is apparently embarrassed by the', "'The jinx technically wouldn’t apply to her,'", '"I can’t say that I’m too upset that Star Wars the Force awaken', '"I cant believe you said this, Blah Blah Blah and wed look', 'People between the ages 15 to 30 are in the risk age range and are most likely to', 'a miami man was arrested sunday after police said he burst into his', '"It broke my heart," said "A little prayer and ate dinner together." "', 'A little girl named Aubrey, one of Lisa and gene’s adopted children', 'There are lots of things that can throw you off your nighttime game if you have regular', '"The capeside gang will likely never get back together," according to Dawsons', 'She understood firsthand the incredible bond between multiples', '"This after all is the man who won five titles for La and was still engaged to La', 'The muse

In [92]:
# type(new_df['spoiler'][0:40])

In [ ]:
# type(stories_list[0:40])

list

In [ ]:
# stories_list_series = pd.Series((v for v in stories_list))

In [65]:
dataframe = pd.DataFrame({ 'postText': new_df_validation["postText"], 'actual_spoiler':test_dataset['labels'], 'gen_spoiler':stories_list })

In [66]:
dataframe

,postText,actual_spoiler,gen_spoiler
0,five nights at freddy ’ s sequel delayed for w...,some of the plot elements are so disturbing th...,Five nights at freddy’s sister location is sch...
4,a man swallowed a microsd card and you wont be...,a man who swallowed a 64GB microSD card and th...,The story is written in the first person by Jo...
7,you wont believe this stunning harry potter re...,McGonagall was appointed as Dumbledore’s assis...,"'The jinx technically wouldn’t apply to her,'"
8,jj abrams has an answer on if there will be a ...,All the scenes are actually in the movie,"""I can’t say that I’m too upset that Star Wars..."
9,kristin cavallari kristincav opens up about ju...,"""I had fake relationships, fake fights. I don'...","""I cant believe you said this, Blah Blah Blah ..."
...,...,...,...
792,one guy decided to explore this abandoned buil...,"This was clearly a crack house, and he was sta...",It’s never the same thing twice though you rea...
793,heres a controversial idea about chinas econom...,China is too quick to rebalance its services s...,China is attempting to move from an economy ba...
794,dog dies one hour after hiking with his owner ...,the plant the dog was chewing on was deadly wa...,The plant the dog was chewing on was deadly wa...
795,this is what happens when you leave a hotel cl...,The video below shows the stunned cleaner init...,a cleaner was met with a stack of cash when sh...


In [68]:
file = open("T5_conditional_finetuning_no_tags_passage_results_without_postText.txt", "w")

#convert variable to string
# str_1 = repr(dataframe)
str2 = dataframe.to_json()
file.write("T5_conditional_finetuning_no_tags_passage_results = " + str2 + "\n")

#close file
file.close()

In [67]:
if WANDB_INTEGRATION:
    wandb_run.finish()

In [ ]:
#EVALUATION METRICS

In [69]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

In [70]:
bleu_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  score_ref = bleu.sentence_bleu([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"])
  bleu_scores.append(score_ref)



In [71]:
bleu_scores

[0.14287998638352478,
 0.15795778058770352,
 0.07741445428076164,
 0.11771058436945724,
 0.1611210171372284,
 0.1449968803035602,
 0.2347269991974008,
 0.1049867194671487,
 0.20419707594116016,
 0.1370761916473635,
 0.09049262147427745,
 0.097944420898446,
 0.2693426731141385,
 0.18655429504260376,
 0.10856261125845283,
 0.16998440083449728,
 0.42846301121432917,
 2.4948060378908767e-78,
 0.2164452446405898,
 0.07053737966973074,
 0.1328391805653375,
 0.15711339157776394,
 0.2986548695866435,
 0.12819475473476505,
 0.16926041430206573,
 6.6578162263375404e-155,
 0.11128290681782532,
 0.16062268845273772,
 0.10330693163689585,
 3.7368367408305776e-78,
 0.10584316886197581,
 0.15923925762996077,
 0.2723453954836604,
 0.11528603723123482,
 0.09193335566142918,
 0.1629644501480937,
 0.13499300144737625,
 3.4508688141984883e-78,
 0.23569627331801513,
 0.4188810544825697,
 0.11095720780169288,
 2.1736124859031784e-78,
 0.1355926756811171,
 0.10255960238310509,
 0.12088075044221112,
 0.200590

In [72]:
bleu_average = round(sum(bleu_scores) / len(bleu_scores), 2)
bleu_average

0.18

In [73]:
bleu_min = min(bleu_scores)
bleu_min

0

In [74]:
bleu_max = max(bleu_scores)
bleu_max

0.9754554449993442

In [75]:
bleu_scores.sort()
mid = len(bleu_scores) // 2
res = (bleu_scores[mid] + bleu_scores[~mid]) / 2
bleu_median = res
bleu_median

0.1487396385478104

In [76]:
import sys
from nltk.translate.meteor_score import meteor_score

In [77]:
meteor_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"].strip()

  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(dataframe.iloc[i]["actual_spoiler"].strip())
  meteor = nltk.translate.meteor_score.meteor_score([ref], hypo)


  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  meteor_scores.append(meteor)



In [78]:
meteor_scores

[0.0,
 0.02824858757062147,
 0.08695652173913045,
 0.029411764705882353,
 0.136986301369863,
 0.02824858757062147,
 0.11718749999999999,
 0.07575757575757576,
 0.09740259740259741,
 0.1092896174863388,
 0.034482758620689655,
 0.0,
 0.0546448087431694,
 0.14492753623188406,
 0.07936507936507936,
 0.09146341463414633,
 0.875742702144149,
 0.0,
 0.03787878787878788,
 0.0,
 0.18244219653179192,
 0.06622516556291393,
 0.04098360655737705,
 0.08333333333333333,
 0.2545362903225806,
 0.07352941176470587,
 0.11029411764705882,
 0.21882352941176472,
 0.23038321167883213,
 0.05847953216374269,
 0.10810810810810811,
 0.05917159763313609,
 0.16089860352155436,
 0.0,
 0.034722222222222224,
 0.22962962962962966,
 0.1,
 0.029585798816568046,
 0.12345679012345678,
 0.648120818575364,
 0.06289308176100629,
 0.07936507936507937,
 0.06666666666666667,
 0.08333333333333333,
 0.09345794392523364,
 0.12987012987012989,
 0.05847953216374269,
 0.10256410256410255,
 0.5484040951520626,
 0.2078212290502793,
 0.

In [79]:
meteor_scores_average = round(sum(meteor_scores) / len(meteor_scores), 2)
meteor_scores_average

0.18

In [80]:
meteor_scores_max = max(meteor_scores)
meteor_scores_max

0.9921405664212475

In [81]:
meteor_scores_min = min(meteor_scores)
meteor_scores_min

0.0

In [82]:
meteor_scores.sort()
mid = len(meteor_scores) // 2
res = (meteor_scores[mid] + meteor_scores[~mid]) / 2
meteor_scores_median = res
meteor_scores_median

0.09360397196261683

In [83]:
drive.flush_and_unmount()

In [ ]:
!pip install bert-score

In [ ]:
from bert_score import score

In [ ]:
!git clone https://github.com/Tiiiger/bert_score.git

Cloning into 'bert_score'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (355/355), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 993 (delta 219), reused 322 (delta 198), pack-reused 638
Receiving objects: 100% (993/993), 1.34 MiB | 10.33 MiB/s, done.
Resolving deltas: 100% (534/534), done.


In [86]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [87]:
!pip install bert_score
bertscore_metric = load_metric('bertscore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
dataframe.iloc[0]

postText          five nights at freddy ’ s sequel delayed for w...
actual_spoiler    some of the plot elements are so disturbing th...
gen_spoiler       Five nights at freddy’s sister location is sch...
Name: 0, dtype: object

In [99]:
dataframe.count()[0]

322

In [116]:
bert_scores_all = []
for i in range(dataframe.count()[0]):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]

  ref = gen_spoiler_strip
  hypo = actual_spoiler_strip

  counter = 0
  for c in hypo: # traverse the string “educative”
    counter+=1 #increment the counter
  # print (counter)

  # print(len(hypo))
  # print("\n")

  counter2 = 0
  for c in ref: # traverse the string “educative”
    counter2+=1 #increment the counter
  # print (counter2)

  len1 = min(counter, counter2)
  # print(hypo[:len1])
  # print("\n")

  if len1 > 0:
    bert_scores = bertscore_metric.compute(predictions=hypo[:len1], references=ref[:len1], lang="en")
    bert_scores_all.append(bert_scores)
  else:
    bert_scores_all.append(0)
# print(bert_scores)
# Normally, we use the f1-score attribute
# print(bert_scores['f1'])

In [117]:
bert_scores_all

[{'precision': [0.9832678437232971,
   0.9987714886665344,
   0.9982216358184814,
   1.0,
   0.0,
   0.9939030408859253,
   0.8613161444664001,
   0.0,
   0.9445218443870544,
   0.9445218443870544,
   0.925399124622345,
   0.0,
   0.9984025955200195,
   0.998131275177002,
   0.0,
   0.8656901121139526,
   0.0,
   1.0,
   0.9997287392616272,
   0.8948938250541687,
   0.9748901128768921,
   0.8689295649528503,
   0.9919165968894958,
   0.0,
   0.9999998807907104,
   0.0,
   0.9831194877624512,
   0.9988217949867249,
   1.0,
   0.0,
   0.0,
   0.9988486766815186,
   0.0,
   0.9918761253356934,
   0.9987497329711914,
   0.9857532382011414,
   0.9975723028182983,
   0.9993895888328552,
   0.9935994148254395,
   0.0,
   0.9999998807907104,
   0.9919165968894958,
   0.0,
   0.0,
   0.9943549036979675,
   0.9999999403953552,
   0.8955549597740173,
   0.9983527660369873,
   0.0,
   0.998131275177002,
   0.9495258331298828,
   0.8948938250541687,
   0.0,
   0.0,
   0.9983219504356384,
   0.0,
  

In [144]:
# type(bert_scores_all)
counter2 = 0
for i in bert_scores_all:
 
    # incrementing counter
    counter2 = counter2 + 1

counter2

322

In [155]:
bert_sum = 0
# counter = 0
for i in range(counter2 - 1):
  berts_all = bert_scores_all[i]
  # print(berts_all["f1"])
  # print("\n")
  # count = 0
  if isinstance(berts_all, dict):
    if "f1" in berts_all:
      for b in berts_all["f1"]:
        # print(b)
        # print("\n")
        bert_sum += b
      #   counter = counter + 1
  # else:
  #   print "Key does not exist"

  # print(len(berts_all["f1"]))
#   for berts in range(len(berts_all["f1"])):

#     bert_sum += berts
#     counter += counter

bertscore_average = round(bert_sum / (counter2 -1), 2)
print(bertscore_average)

44.94


In [ ]:



bert_scores_R

In [ ]:
bert_scores_F1